# IMPORTS

In [1]:
import tensorflow as tf
from sklearn.metrics import precision_recall_curve
import numpy as np
import os
from tensorflow.keras.metrics import Precision, Recall, AUC
from sklearn.metrics import f1_score, classification_report
import json
from google.colab import runtime
from google.colab import drive
drive.mount('/content/drive')
import gc

Mounted at /content/drive


# DATASET

In [2]:
batch_size = 1
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
categories = ["Normal","Osteopenia", "Osteoporosis"]
TfDatasetsPath = '/content/drive/MyDrive/CSEN-240-Project/TfDatasetsFull'

valDataset = tf.data.Dataset.load(TfDatasetsPath + "/validDataset")
testDataset = tf.data.Dataset.load(TfDatasetsPath + "/testDataset")

valDataset = valDataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
testDataset = testDataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

# MODELS

In [3]:
thresholdsPath = '/content/drive/MyDrive/CSEN-240-Project/Thresholds/'
kerasModelsPath = '/content/drive/MyDrive/CSEN-240-Project/KerasModels/'

In [4]:
modelNames = os.listdir(kerasModelsPath)
modelNames

['Xception.keras',
 'VGG16.keras',
 'ResNet152V2.keras',
 'DenseNet201.keras',
 'InceptionResNetV2.keras',
 'EfficientNetV2L.keras',
 'NASNetLarge.keras',
 'MobileNet.keras']

In [5]:
models = []
for modelName in modelNames:
    modelPath = kerasModelsPath + modelName
    model = tf.keras.models.load_model(modelPath)
    results = model.evaluate(valDataset, return_dict=True)
    models.append([modelName, results['auc']])
models

1080/1080 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - accuracy: 0.8897 - auc: 0.9595 - loss: 0.4721 - precision: 0.8906 - recall: 0.8885
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - accuracy: 0.8891 - auc: 0.9684 - loss: 0.3453 - precision: 0.8901 - recall: 0.8839
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 36s 18ms/step - accuracy: 0.8429 - auc: 0.9464 - loss: 0.5786 - precision: 0.8424 - recall: 0.8382
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 56s 20ms/step - accuracy: 0.8921 - auc: 0.9644 - loss: 0.4197 - precision: 0.8929 - recall: 0.8909
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 42s 19ms/step - accuracy: 0.8796 - auc: 0.9704 - loss: 0.3603 - precision: 0.8822 - recall: 0.8780
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 62s 28ms/step - accuracy: 0.8667 - auc: 0.9558 - loss: 0.4460 - precision: 0.8707 - recall: 0.8594
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 72s 37ms/step - accuracy: 0.8454 - auc: 0.9465 - loss: 0.5670 - precision: 0.8462 - recall: 0.8430
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - accuracy: 0.8460 - auc: 0.9498 - loss: 0.4936 

[['Xception.keras', 0.9669954180717468],
 ['VGG16.keras', 0.9701573252677917],
 ['ResNet152V2.keras', 0.9537114500999451],
 ['DenseNet201.keras', 0.9715805649757385],
 ['InceptionResNetV2.keras', 0.9750535488128662],
 ['EfficientNetV2L.keras', 0.9626356959342957],
 ['NASNetLarge.keras', 0.9577512145042419],
 ['MobileNet.keras', 0.9579649567604065]]

In [6]:
models.sort(key=lambda x: x[1], reverse=True)
modelNames = [model[0] for model in models]
modelNames

['InceptionResNetV2.keras',
 'DenseNet201.keras',
 'VGG16.keras',
 'Xception.keras',
 'EfficientNetV2L.keras',
 'MobileNet.keras',
 'NASNetLarge.keras',
 'ResNet152V2.keras']

# ENSEMBLING FUNCTION

In [7]:
def ensembleModels(predictions):
    avgPreds = np.mean(predictions, axis=0)
    return avgPreds

In [8]:
def predict(modelNames, testDataset):
    predictions = []
    for modelName in modelNames:
        modelPath = kerasModelsPath + modelName
        model = tf.keras.models.load_model(modelPath)
        prediction = model.predict(testDataset)
        predictions.append(prediction)
        tf.keras.backend.clear_session(free_memory=True)
        del model
        gc.collect()
    avgPreds = ensembleModels(predictions)
    predictedClasses = np.argmax(avgPreds, axis=1)
    return predictedClasses, avgPreds

In [9]:
bestAverageF1 = -1
topK = -1
bestModels = None

yTrue = []
for xBatch, yBatch in testDataset:
    yTrue.extend(np.argmax(yBatch.numpy(), axis=1))

for k in range(2, len(modelNames)+1):
    print(f"Top {k} : {modelNames[:k]}")
    yPred, predProbs = predict(modelNames[:k], testDataset)
    f1 = f1_score(yTrue, yPred, average='weighted')
    if f1 > bestAverageF1:
        bestAverageF1 = f1
        topK = k
        bestModels = modelNames[:k]
    report = classification_report(yTrue, yPred, target_names=categories)
    print("F1 Score: ", f1)
    print(report)
bestAverageF1, topK, bestModels

Top 2 : ['InceptionResNetV2.keras', 'DenseNet201.keras']
540/540 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step
540/540 ━━━━━━━━━━━━━━━━━━━━ 31s 21ms/step
F1 Score:  0.9241927641927642
              precision    recall  f1-score   support

      Normal       0.90      0.95      0.92       180
  Osteopenia       0.96      0.91      0.94       180
Osteoporosis       0.91      0.91      0.91       180

    accuracy                           0.92       540
   macro avg       0.93      0.92      0.92       540
weighted avg       0.93      0.92      0.92       540

Top 3 : ['InceptionResNetV2.keras', 'DenseNet201.keras', 'VGG16.keras']
540/540 ━━━━━━━━━━━━━━━━━━━━ 22s 19ms/step
540/540 ━━━━━━━━━━━━━━━━━━━━ 30s 21ms/step
540/540 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step
F1 Score:  0.9315203929088783
              precision    recall  f1-score   support

      Normal       0.92      0.96      0.94       180
  Osteopenia       0.97      0.91      0.94       180
Osteoporosis       0.91      0.93      0.92       180


(0.9352178733478992,
 4,
 ['InceptionResNetV2.keras',
  'DenseNet201.keras',
  'VGG16.keras',
  'Xception.keras'])

# GETTING BEST THRESHOLDS

In [10]:
predictions = []
for modelName in bestModels:
    modelPath = kerasModelsPath + modelName
    model = tf.keras.models.load_model(modelPath)
    prediction = model.predict(valDataset)
    predictions.append(prediction)
    tf.keras.backend.clear_session(free_memory=True)
    del model
    gc.collect()
ensemblePredictions = ensembleModels(predictions)

1080/1080 ━━━━━━━━━━━━━━━━━━━━ 33s 19ms/step
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 41s 21ms/step
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step


In [11]:
def bestThresholds(predictions, valDataset):
    valid_labels = []
    for xBatch, yBatch in valDataset:
        valid_labels.extend(np.argmax(yBatch, axis=1))
    yTrue = np.eye(3)[valid_labels]
    yPred = predictions
    thresholds = {}

    for class_idx in range(3):
        precision, recall, thresholds_pr = precision_recall_curve(yTrue[:, class_idx], yPred[:, class_idx])

        f1_scores = 2 * (precision * recall) / (precision + recall)

        optimal_threshold_pr = thresholds_pr[np.argmax(f1_scores)]
        thresholds[class_idx] = optimal_threshold_pr

    return thresholds

In [12]:
thresholds = bestThresholds(ensemblePredictions, valDataset)

In [13]:
predictions = []
for modelName in bestModels:
    modelPath = kerasModelsPath + modelName
    model = tf.keras.models.load_model(modelPath)
    prediction = model.predict(testDataset)
    predictions.append(prediction)
    tf.keras.backend.clear_session(free_memory=True)
    del model
    gc.collect()
ensemblePredictions = ensembleModels(predictions)

540/540 ━━━━━━━━━━━━━━━━━━━━ 22s 20ms/step
540/540 ━━━━━━━━━━━━━━━━━━━━ 29s 21ms/step
540/540 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step
540/540 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step


In [14]:
def customPredict(yPred, thresholds):
    predictions = np.zeros_like(yPred)
    for i in thresholds.keys():
        threshold = thresholds[i]
        predictions[:, i] = (yPred[:, i] >= threshold).astype(int)
    predictions = np.argmax(predictions * yPred, axis=1)
    return predictions

In [15]:
yTrue = []
for xBatch, yBatch in testDataset:
    yTrue.extend(np.argmax(yBatch.numpy(), axis=1))

yPred = customPredict(ensemblePredictions, thresholds)

report = classification_report(yTrue, yPred, target_names=categories)
print(report)

              precision    recall  f1-score   support

      Normal       0.92      0.97      0.95       180
  Osteopenia       0.97      0.94      0.95       180
Osteoporosis       0.93      0.91      0.92       180

    accuracy                           0.94       540
   macro avg       0.94      0.94      0.94       540
weighted avg       0.94      0.94      0.94       540



In [16]:
runtime.unassign()